In [ ]:
import oracledb, os, json
from google.cloud import secretmanager
import pandas as pd
import numpy as np
import paramiko
from io import StringIO
from datetime import date
from datetime import timedelta
import datetime

In [ ]:
def get_periode():
    """
    henter periode for the tidligere måneden eksample--> i dag er 19.04.2022, metoden vil kalkulerer periode aarMaaned eks) '202203'
    :param periode:
    :return: periode
    """
    today = datetime.date.today() # dato for idag 2022-04-19
    first = today.replace(day=1) # dato for første dag i måneden 2022-04-01
    lastMonth = first - datetime.timedelta(days=1) # dato for siste dag i tidligere måneden

    return lastMonth.strftime("%Y%m") # henter bare aar og maaned

periode = get_periode()

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    sftpkey = os.getenv('SFTPKEY'),
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
keyfile = StringIO(oracle_secrets['sftpkey'])
mykey = paramiko.RSAKey.from_private_key(keyfile, password=np.nan)

# Open a transport
host,port = "a01drvl099.adeo.no",22
transport = paramiko.Transport((host,port))

# Auth    
username= "srv-dv-familie-airflow-sas"
transport.connect(username=username,pkey=mykey)

with paramiko.SFTPClient.from_transport(transport) as sftp:
    sftp.get(f'./inbound/kildefiler/bidrag/BIDRAG_STONAD_M{periode}', f'../data/BIDRAG_STONAD_M{periode}')

# Close
if sftp: sftp.close()
if transport: transport.close()

In [ ]:
%%bash

cat ../data/BIDRAG_STONAD_M$periode | tr -d '\000' > ../data/BIDRAG_STONAD_M$periode.txt

In [ ]:
df = pd.read_csv(f"../data/BIDRAG_STONAD_M2024{periode}.txt", skip_blank_lines=True, sep='\0', header=None)
df['RECTY'] = df[0].apply(lambda x: x[13:15])

filt = (df['RECTY'] == '02')
df_RECTY_2 = df.loc[filt].copy()

kolonne_navn = ["@1   PERIODE    6.","@7 SAKSNR     $7.",  "@14 RECTY     2.", "@7 NR         $2.",  "@16 TKNR     $4.",  "@16  FYLKE   $2. ",             
"@20 BMFDAG    $2.",  "@22 BMFMND   $2.",  "@24  BMFAAR  $2. ", "@20 BMFNR     $11.", "@31 ALDER_BM 2.",  " @33  KJONN   $1.", "@34 BOLAND  $3.", "@37 STONSTAT  7.",  "@44 SAKSTYP   $1.",  "@45  EKBIDRG 7.  ", "@52 OBJNR    2.",   
"@54 ANTBARNB  3.",  "@57 EKBIDR     7.",  " @64  BIDRSUM 7. ", " @71 LOPSAK   $1."  ,"@72 UTLVAL    $3."]

for i in kolonne_navn:
    m = i.split()
    kolonnne = m[1]
    start_pos = int("".join([str(s) for s in m[0] if s.isdigit()])) - 1
    end_pos = int("".join([str(s) for s in m[2] if s.isdigit()])) + start_pos
    df_RECTY_2[kolonnne] = df_RECTY_2[0].apply(lambda x: x[start_pos:end_pos])

df_mottaker_pos = df_RECTY_2[['PERIODE','RECTY','SAKSNR','NR','TKNR','FYLKE','BMFDAG','BMFMND','BMFAAR','BMFNR','ALDER_BM','KJONN',
'BOLAND', 'STONSTAT','SAKSTYP','EKBIDRG','OBJNR','EKBIDR','BIDRSUM','LOPSAK','UTLVAL']]
df_mottaker_pos = df_mottaker_pos.replace(r'^\s*$', None, regex=True)

# filtrere vekk rader der ALDER_BM = 10 (det er de som har ALDER_BM >= 100)
filt = ((df_mottaker_pos['ALDER_BM'] != '10') & (df_mottaker_pos['ALDER_BM'] != '11'))
df_mottaker_pos = df_mottaker_pos.loc[filt]

user = oracle_secrets['user'] + '[DVH_FAM_BB]'
dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])

with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
    with conn.cursor() as cursor:
        rows = [tuple(x) for x in df_mottaker_pos.values]
        cursor.executemany('''INSERT INTO FAM_BB_mottaker_BIS (PERIODE,RECTY,SAKSNR,NR,TKNR,FYLKE,BMFDAG,BMFMND,BMFAAR,BMFNR,ALDER_BM,KJONN,
                                BOLAND, STONSTAT,SAKSTYP,EKBIDRG,OBJNR,EKBIDR,BIDRSUM,LOPSAK,UTLVAL) 
                                VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21)''',rows)
        conn.commit()
